# 8. Generate Evaluation Result

We have defined 9 queries to search over the evaluation dataset (products with 11-14 reviews). The output was then rated by 3 raters with a scale of 1 to 5 where 1 denoted “Not relevant at all” while 5 denoted “Perfectly relevant”. 

We use [Normalized Discounted Cumulative Gain (NDCG)](https://en.wikipedia.org/wiki/Discounted_cumulative_gain) to evaluate the goodness of ranking for our search engine under the 3 ranking methods, i.e. *i) Average*, *ii) Discounted Reward*, and *iii) Discounted Reward with Adjustment by Opposite Query*. The *i) Average* method acts as the baseline for our evaluation.

In [1]:
# Import libraries.
import pandas as pd
import glob
import math
import krippendorff
from sklearn.metrics import ndcg_score
import pingouin as pg
import session_info

In [2]:
# Initialize paths for files containing product-level ratings from 1-5, 5 being the most relevant.
u1_rating_file_path='../resources/eval/Ratings_U1/*.xlsx'
u2_rating_file_path='../resources/eval/Ratings_U2/*.xlsx'
u3_rating_file_path='../resources/eval/Ratings_U3.xlsx'

## Calculating NDCG

For the details on how NDCG is calculated, we recommend this article [“Demystifying NDCG” by Aparna Dhinakaran](https://towardsdatascience.com/demystifying-ndcg-bee3be58cfe0).   
Here for our implementation, we use the `ndcg_score` function from [sklearn](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.ndcg_score.html). 

In [3]:
# Function to calculate Normalized Discounted Cumulative Gain for each query, each ranking method, and each user.
def NDCG(u1_rating_file_path, u2_rating_file_path, u3_rating_file_path):
    
    # Create file lists for Users 1 and 2 since they have similar rating file formats.
    u1_file_paths=glob.glob(u1_rating_file_path)
    u2_file_paths=glob.glob(u2_rating_file_path)
    file_paths=[u1_file_paths, u2_file_paths]
    user_df_dict={}
    
    # Loop through each file in the folders for Users 1 and 2.
    for file_path in file_paths:
        ndcg_dict={}
        for path in file_path:
            
            # Read each rating file as a dataframe and perform basic sanity checks.
            df=pd.read_excel(path)
            df.dropna(subset=["Query ID", "User ID"], inplace=True)
            # Ensure that each line item has product-level rating.
            df=df.loc[~(df["Detailed Rating (product level)"].isna())|(~(df["Relevance Indicator"].isna()))]
            # Ensure that the relevance indicator is present for at least one product for a given query and ranking method.
            df.loc[(df["product_id"].duplicated(keep=False))&(~(df["Detailed Rating (product level)"].isna())), "Relevance Indicator"]="Most relevant"
            df.dropna(subset=["Detailed Rating (product level)"], inplace=True)
            
            # Calculate the ideal rank
            df["ideal_rating"]=sorted(list(df["Detailed Rating (product level)"]), reverse=True)
  
            # Calculate the NDCG
            ndcg_10 = ndcg_score(df['ideal_rating'].values.reshape(1,-1), df['Detailed Rating (product level)'].values.reshape(1,-1), k=10)
            ndcg_5 = ndcg_score(df['ideal_rating'].values.reshape(1,-1), df['Detailed Rating (product level)'].values.reshape(1,-1), k=5)
            ndcg_3 = ndcg_score(df['ideal_rating'].values.reshape(1,-1), df['Detailed Rating (product level)'].values.reshape(1,-1), k=3)
            
            # Add relevant information as keys and values in a dictionary.
            if "Query ID" not in ndcg_dict.keys():
                ndcg_dict["Query ID"]=[df["Query ID"].iloc[0]]
            else:
                ndcg_dict["Query ID"].append(df["Query ID"].iloc[0])
            if "Ranking Method" not in ndcg_dict.keys():
                ndcg_dict["Ranking Method"]=[df["Ranking Method"].iloc[0]]
            else:
                ndcg_dict["Ranking Method"].append(df["Ranking Method"].iloc[0])
            if "User ID" not in ndcg_dict.keys():
                ndcg_dict["User ID"]=[df["User ID"].iloc[0]]
            else:
                ndcg_dict["User ID"].append(df["User ID"].iloc[0])
            if "NDCG@10" not in ndcg_dict.keys():
                ndcg_dict["NDCG@10"]=[ndcg_10]
            else:
                ndcg_dict["NDCG@10"].append(ndcg_10)
            if "NDCG@5" not in ndcg_dict.keys():
                ndcg_dict["NDCG@5"]=[ndcg_5]
            else:
                ndcg_dict["NDCG@5"].append(ndcg_5)
            if "NDCG@3" not in ndcg_dict.keys():
                ndcg_dict["NDCG@3"]=[ndcg_3]
            else:
                ndcg_dict["NDCG@3"].append(ndcg_3)

        # Convert the dictionary to a dataframe and tidy it up.
        u_df=pd.DataFrame(ndcg_dict)
        u_df1=u_df.loc[u_df["Query ID"]!="Q10"].copy()
        u_df2=u_df.loc[u_df["Query ID"]=="Q10"].copy()
        u_df=pd.concat([u_df1, u_df2])
        u_df.reset_index(drop=True, inplace=True)
        # Add each user's result dataframe as a value in a dictionary.
        user_df_dict[df["User ID"].iloc[0]]=u_df
    
    # Create a list of query IDs.
    queries=list(u_df["Query ID"].unique())
    
    # Loop through each query ID for User 3, whose rating file differs from those of Users 1 and 2.
    ndcg_dict={}
    for query in queries:
        
        # Read each sheet in the rating file as a dataframe and perform basic sanity checks.
        df=pd.read_excel(u3_rating_file_path, sheet_name=query)
        df.dropna(subset=["product_id","review_id"], inplace=True)
        
        # Separate the dataframe based on ranking method for ease of calculation.
        dfa=df.loc[df["Ranking Method"]=="Average"].copy()
        dfd=df.loc[df["Ranking Method"]=="Discounted Reward Only"].copy()
        dfda=df.loc[df["Ranking Method"]=="Discounted Reward with Adjustment"].copy()

        # Create a list of the dataframes corresponding to each ranking method.
        df_list=[dfa, dfd, dfda]

        # Loop through each dataframe in the list corresponding to a different ranking method.
        for dfx in df_list:
            # Calculate the ideal rank
            dfx["ideal_rating"]=sorted(list(dfx["Detailed Rating (product level)"]), reverse=True)

            # Calculate the NDCG
            ndcg_10 = ndcg_score(dfx['ideal_rating'].values.reshape(1,-1), dfx['Detailed Rating (product level)'].values.reshape(1,-1), k=10)
            ndcg_5 = ndcg_score(dfx['ideal_rating'].values.reshape(1,-1), dfx['Detailed Rating (product level)'].values.reshape(1,-1), k=5)
            ndcg_3 = ndcg_score(dfx['ideal_rating'].values.reshape(1,-1), dfx['Detailed Rating (product level)'].values.reshape(1,-1), k=3)

            # Add relevant information as keys and values in a dictionary.
            if "Query ID" not in ndcg_dict.keys():
                ndcg_dict["Query ID"]=[dfx["Query ID"].iloc[0]]
            else:
                ndcg_dict["Query ID"].append(dfx["Query ID"].iloc[0])
            if "Ranking Method" not in ndcg_dict.keys():
                ndcg_dict["Ranking Method"]=[dfx["Ranking Method"].iloc[0]]
            else:
                ndcg_dict["Ranking Method"].append(dfx["Ranking Method"].iloc[0])
            if "User ID" not in ndcg_dict.keys():
                ndcg_dict["User ID"]=[dfx["User ID"].iloc[0]]
            else:
                ndcg_dict["User ID"].append(dfx["User ID"].iloc[0])
            if "NDCG@10" not in ndcg_dict.keys():
                ndcg_dict["NDCG@10"]=[ndcg_10]
            else:
                ndcg_dict["NDCG@10"].append(ndcg_10)
            if "NDCG@5" not in ndcg_dict.keys():
                ndcg_dict["NDCG@5"]=[ndcg_5]
            else:
                ndcg_dict["NDCG@5"].append(ndcg_5)
            if "NDCG@3" not in ndcg_dict.keys():
                ndcg_dict["NDCG@3"]=[ndcg_3]
            else:
                ndcg_dict["NDCG@3"].append(ndcg_3)

    # Convert the dictionary to a dataframe and tidy it up.
    u_df=pd.DataFrame(ndcg_dict)
    # Add User 3's result dataframe as a value in the dictionary that contains the result dataframes for Users 1 and 2.
    user_df_dict[u_df["User ID"].iloc[0]]=u_df
    
    # Concatenate all three users' result dataframes into one big dataframe.
    user_df=pd.DataFrame()
    for df in user_df_dict.keys():
        user_df=pd.concat([user_df, user_df_dict[df]])

    # Return the dictionary containing each user's result dataframes and the big dataframe containing all 3 users' NDCG results.
    return user_df_dict, user_df

In [4]:
df_ndcg_by_users, df_ndcg = NDCG(u1_rating_file_path, u2_rating_file_path, u3_rating_file_path)

In [5]:
# Sample output
df_ndcg_by_users['U2'].head(10)

,Query ID,Ranking Method,User ID,NDCG@10,NDCG@5,NDCG@3
0,Q1,Average,U2,0.926717,0.855328,0.796736
1,Q1,Discounted Reward Only,U2,0.946090,0.838319,0.856825
2,Q1,Discounted Reward with Adjustment,U2,0.945335,0.824223,0.821877
3,Q2,Average,U2,0.938423,0.833333,0.833333
4,Q2,Discounted Reward Only,U2,0.938423,0.833333,0.833333
5,Q2,Discounted Reward with Adjustment,U2,0.933606,0.820920,0.833333
6,Q3,Average,U2,0.963934,0.922161,0.898368
7,Q3,Discounted Reward Only,U2,0.958255,0.901201,0.887982
8,Q3,Discounted Reward with Adjustment,U2,0.957288,0.897008,0.885905
9,Q4,Average,U2,0.904810,0.800183,0.711756


In [6]:
# uncomment below line if you want to save the result
#df_ndcg.to_csv('../resources/eval/ndcg.csv', index=False)

Refer to the notebook `evaluation_analysis.ipynb` for further analysis on NDCG.

## Calculate Mean Reciprocal Rank (MRR)

MRR is another measurement of the goodness of ranking by measuring how far down the ranking the first relevant document is.  
For details, please visit this article - [Compute Mean Reciprocal Rank (MRR) using Pandas](https://softwaredoug.com/blog/2021/04/21/compute-mrr-using-pandas.html)

In [7]:
# Function to calculate Mean Reciprocal Rank for each ranking method and each user.
def MRR(u1_rating_file_path, u2_rating_file_path, u3_rating_file_path):
    
    # Create file lists for Users 1 and 2 since they have similar rating file formats.
    u1_file_paths=glob.glob(u1_rating_file_path)
    u2_file_paths=glob.glob(u2_rating_file_path)
    file_paths=[u1_file_paths, u2_file_paths]
    user_df_dict={}
    
    # Loop through each file in the folders for Users 1 and 2.
    for file_path in file_paths:
        mrr_dict={}
        for path in file_path:
            
            # Read each rating file as a dataframe and perform basic sanity checks.
            df=pd.read_excel(path)
            df.dropna(subset=["Query ID", "User ID"], inplace=True)
            # Ensure that each line item has product-level rating.
            df=df.loc[~(df["Detailed Rating (product level)"].isna())|(~(df["Relevance Indicator"].isna()))]
            # Ensure that the relevance indicator is present for at least one product for a given query and ranking method.
            df.loc[(df["product_id"].duplicated(keep=False))&(~(df["Detailed Rating (product level)"].isna())), "Relevance Indicator"]="Most relevant"
            df.dropna(subset=["Detailed Rating (product level)"], inplace=True)
            
            # Calculate the rank, the reciprocal rank, and the reciprocal rank of the most relevant product.
            df["rank"]=list(range(1, len(df)+1))
            df["reciprocal_rank"]=round(1/df["rank"], 3)
            rr=df.loc[df["Relevance Indicator"]=="Most relevant", "reciprocal_rank"].iloc[0]
            
            # Add relevant information as keys and values in a dictionary.
            if "Query ID" not in mrr_dict.keys():
                mrr_dict["Query ID"]=[df["Query ID"].iloc[0]]
            else:
                mrr_dict["Query ID"].append(df["Query ID"].iloc[0])
            if "Ranking Method" not in mrr_dict.keys():
                mrr_dict["Ranking Method"]=[df["Ranking Method"].iloc[0]]
            else:
                mrr_dict["Ranking Method"].append(df["Ranking Method"].iloc[0])
            if "User ID" not in mrr_dict.keys():
                mrr_dict["User ID"]=[df["User ID"].iloc[0]]
            else:
                mrr_dict["User ID"].append(df["User ID"].iloc[0])
            if "Reciprocal Rank" not in mrr_dict.keys():
                mrr_dict["Reciprocal Rank"]=[rr]
            else:
                mrr_dict["Reciprocal Rank"].append(rr)
        
        # Convert the dictionary to a dataframe and tidy it up.
        u_df=pd.DataFrame(mrr_dict)
        u_df1=u_df.loc[u_df["Query ID"]!="Q10"].copy()
        u_df2=u_df.loc[u_df["Query ID"]=="Q10"].copy()
        u_df=pd.concat([u_df1, u_df2])
        u_df.reset_index(drop=True, inplace=True)
        # Add each user's result dataframe as a value in a dictionary.
        user_df_dict[df["User ID"].iloc[0]]=u_df
        
    # Create a list of query IDs.
    queries=list(u_df["Query ID"].unique())
    
    # Loop through each query ID for User 3, whose rating file format differs from those of Users 1 and 2.
    mrr_dict={}
    for query in queries:
        
        # Read each sheet in the rating file as a dataframe and perform basic sanity checks.
        df=pd.read_excel(u3_rating_file_path, sheet_name=query)
        df.dropna(subset=["product_id","review_id"], inplace=True)

        # Separate the dataframe based on ranking method for ease of calculation.
        dfa=df.loc[df["Ranking Method"]=="Average"].copy()
        dfd=df.loc[df["Ranking Method"]=="Discounted Reward Only"].copy()
        dfda=df.loc[df["Ranking Method"]=="Discounted Reward with Adjustment"].copy()

        # Create a list of the dataframes corresponding to each ranking method.
        df_list=[dfa, dfd, dfda]

        # Loop through each dataframe in the list corresponding to a different ranking method.
        for dfx in df_list:
            
            # Calculate the rank, the reciprocal rank, and the reciprocal rank of the most relevant product.           
            dfx["rank"]=list(range(1, len(dfx)+1))
            dfx["reciprocal_rank"]=round(1/dfx["rank"], 3)
            rr=dfx.loc[dfx["Relevance Indicator"]=="Most relevant", "reciprocal_rank"].iloc[0]
            
            # Add relevant information as keys and values in a dictionary.
            if "Query ID" not in mrr_dict.keys():
                mrr_dict["Query ID"]=[dfx["Query ID"].iloc[0]]
            else:
                mrr_dict["Query ID"].append(dfx["Query ID"].iloc[0])
            if "Ranking Method" not in mrr_dict.keys():
                mrr_dict["Ranking Method"]=[dfx["Ranking Method"].iloc[0]]
            else:
                mrr_dict["Ranking Method"].append(dfx["Ranking Method"].iloc[0])
            if "User ID" not in mrr_dict.keys():
                mrr_dict["User ID"]=[dfx["User ID"].iloc[0]]
            else:
                mrr_dict["User ID"].append(dfx["User ID"].iloc[0])
            if "Reciprocal Rank" not in mrr_dict.keys():
                mrr_dict["Reciprocal Rank"]=[rr]
            else:
                mrr_dict["Reciprocal Rank"].append(rr)
                
    # Convert the dictionary to a dataframe and tidy it up.
    u_df=pd.DataFrame(mrr_dict)
    # Add User 3's result dataframe as a value in the dictionary that contains the result dataframes for Users 1 and 2.
    user_df_dict[u_df["User ID"].iloc[0]]=u_df
    
    # Concatenate all three users' result dataframes into one big dataframe.
    user_df=pd.DataFrame()
    for df in user_df_dict.keys():
        user_df=pd.concat([user_df, user_df_dict[df]])
    
    # Calculate the MRR after grouping the result by the Ranking Method for each user.
    u1_ranks=user_df.loc[user_df["User ID"]=="U1"].copy()
    u2_ranks=user_df.loc[user_df["User ID"]=="U2"].copy()
    u3_ranks=user_df.loc[user_df["User ID"]=="U3"].copy()
    u1_mrr=u1_ranks.groupby(["Ranking Method"]).agg({"Reciprocal Rank": "mean"}).rename(columns={"Reciprocal Rank": "MRR"}).reset_index()
    u2_mrr=u2_ranks.groupby(["Ranking Method"]).agg({"Reciprocal Rank": "mean"}).rename(columns={"Reciprocal Rank": "MRR"}).reset_index()
    u3_mrr=u3_ranks.groupby(["Ranking Method"]).agg({"Reciprocal Rank": "mean"}).rename(columns={"Reciprocal Rank": "MRR"}).reset_index()
    u1_mrr["User ID"]="U1"
    u2_mrr["User ID"]="U2"
    u3_mrr["User ID"]="U3"
    
    # Combine the MRR result dataframe of all three users.
    ranks_df=pd.concat([u1_mrr, u2_mrr, u3_mrr])
            
    # Return the dictionary containing each user's result dataframes and the dataframes containing all 3 users' MRR results.
    return user_df_dict, user_df, ranks_df

In [8]:
df_mrr_by_users, df_mrr, df_mrr_ranks = MRR(u1_rating_file_path, u2_rating_file_path, u3_rating_file_path)

In [9]:
# Sample result
df_mrr_ranks

,Ranking Method,MRR,User ID
0,Average,0.568444,U1
1,Discounted Reward Only,0.401778,U1
2,Discounted Reward with Adjustment,0.612111,U1
0,Average,0.394333,U2
1,Discounted Reward Only,0.351333,U2
2,Discounted Reward with Adjustment,0.584333,U2
0,Average,0.494333,U3
1,Discounted Reward Only,0.329222,U3
2,Discounted Reward with Adjustment,0.490667,U3


In [18]:
# uncomment below line if you want to save the result
#df_mrr_ranks.to_csv('../resources/eval/mrr.csv', index=False)

Refer to the notebook `evaluation_analysis.ipynb` for further analysis on MRR.

## Calculate Krippendorff's alpha

Krippendorff's alpha is used to evaluate inter-rater reliability. Where Cohen's kappa measures are used for the same purpose but in cases where only two raters provide ratings, Krippendorff's alpha can be conveniently used in the case of multiple raters.

For details, please visit this article - [Inter-Annotator Agreement: An Introduction to Krippendorff’s Alpha](https://www.surgehq.ai/blog/inter-rater-reliability-metrics-an-introduction-to-krippendorffs-alpha)

In [11]:
# Function to calculate. Krippendorff's alpha to evaluate inter-rater reliability
def Krippendorff_alpha(u1_rating_file_path, u2_rating_file_path, u3_rating_file_path):
    
    # Create file lists for Users 1 and 2 since they have similar rating file formats.
    u1_file_paths=glob.glob(u1_rating_file_path)
    u2_file_paths=glob.glob(u2_rating_file_path)
    file_paths=[u1_file_paths, u2_file_paths]
    user_df_dict={}
    
    # Loop through each file in the folders for Users 1 and 2.
    user_dict={}
    for file_path in file_paths:
        kripp_dict={}
        for path in file_path:
            
            # Read each rating file as a dataframe and perform basic sanity checks.
            df=pd.read_excel(path)
            df.dropna(subset=["Query ID", "User ID"], inplace=True)
            # Ensure that each line item has product-level rating.
            df=df.loc[~(df["Detailed Rating (product level)"].isna())|(~(df["Relevance Indicator"].isna()))]
            # Ensure that the relevance indicator is present for at least one product for a given query and ranking method.
            df.loc[(df["product_id"].duplicated(keep=False))&(~(df["Detailed Rating (product level)"].isna())), "Relevance Indicator"]="Most relevant"
            df.dropna(subset=["Detailed Rating (product level)"], inplace=True)
            
            # Keep relevant columns.
            df=df.loc[:,["Query ID", "Ranking Method", "product_id", "User ID", "Detailed Rating (product level)"]]
            # Create a new column which is a string combination of Query ID, Ranking Method and product_id.
            df["Query ID__Ranking Method__product_id"]=df["Query ID"]+"_"+df["Ranking Method"]+"_"+df["product_id"]
            # Add relevant information as keys and values in a dictionary.
            kripp_dict[df["User ID"].iloc[0]+"_"+df["Query ID"].iloc[0]+"_"+df["Ranking Method"].iloc[0]]=df
            
        # Add each user's dictionary as a value in a dictionary, creating a nested dictionary.
        user_dict[df["User ID"].iloc[0]]=kripp_dict
    
    # Create a list of query IDs.
    queries=["Q1","Q2","Q3","Q4","Q5","Q6","Q7","Q8","Q9"]
    
    # Loop through each query ID for User 3, whose rating file format differs from those of Users 1 and 2.
    kripp_dict={}
    for query in queries:
        
        # Read each sheet in the rating file as a dataframe and perform basic sanity checks.
        df=pd.read_excel(u3_rating_file_path, sheet_name=query)
        df.dropna(subset=["product_id","review_id"], inplace=True)

        # Separate the dataframe based on ranking method for ease of calculation.
        dfa=df.loc[df["Ranking Method"]=="Average"].copy()
        dfd=df.loc[df["Ranking Method"]=="Discounted Reward Only"].copy()
        dfda=df.loc[df["Ranking Method"]=="Discounted Reward with Adjustment"].copy()

        # Create a list of the dataframes corresponding to each ranking method.
        df_list=[dfa, dfd, dfda]

        # Loop through each dataframe in the list corresponding to a different ranking method.
        for dfx in df_list:
            
            # Keep relevant columns.
            dfx=dfx.loc[:,["Query ID", "Ranking Method", "product_id", "User ID", "Detailed Rating (product level)"]]
            # Create a new column which is a string combination of Query ID, Ranking Method and product_id.
            dfx["Query ID__Ranking Method__product_id"]=dfx["Query ID"]+"_"+dfx["Ranking Method"]+"_"+dfx["product_id"]
            # Add relevant information as keys and values in a dictionary.
            kripp_dict[dfx["User ID"].iloc[0]+"_"+dfx["Query ID"].iloc[0]+"_"+dfx["Ranking Method"].iloc[0]]=dfx
    
    # Add User 3's dictionary as a value in the dictionary that contains the result dictionaries for Users 1 and 2.
    user_dict[dfx["User ID"].iloc[0]]=kripp_dict
    
    # Combine all 3 users' dataframes by querying the nested dictionary.
    main_df=pd.DataFrame()
    for user in user_dict.keys():
        for df in user_dict[user]:
            dfx=user_dict[user][df]
            main_df=pd.concat([main_df, dfx])
    
    # Create a list of each user's ratings for Krippendorff's alpha calculation.
    u1_ratings_df=main_df.loc[main_df["User ID"]=="U1"].copy()
    u2_ratings_df=main_df.loc[main_df["User ID"]=="U2"].copy()
    u3_ratings_df=main_df.loc[main_df["User ID"]=="U3"].copy()
    u1_ratings_list=list(u1_ratings_df["Detailed Rating (product level)"])
    u2_ratings_list=list(u2_ratings_df["Detailed Rating (product level)"])
    u3_ratings_list=list(u3_ratings_df["Detailed Rating (product level)"])
        
    # Create a list of lists, with each sublist corresponding to the ratings of one user.
    ratings_list=[u1_ratings_list, u2_ratings_list, u3_ratings_list]

    # Calculate Krippendorff's alpha.
    alpha=round(krippendorff.alpha(reliability_data=ratings_list, level_of_measurement="ordinal"), 3)
    
    # Return the Krippendorff's alpha value.
    return alpha

In [12]:
alpha=Krippendorff_alpha(u1_rating_file_path, u2_rating_file_path, u3_rating_file_path)

In [13]:
alpha

0.585

Krippendorff's alpha is a quantitative measure based on the observed disagreement of raters corrected for disagreement expected by chance. The range of this measure lies between -1 and 1, where 1 indicates perfect agreement, 0 indicates no agreement beyond chance, and negative values indicate inverse agreement.

Our Krippendorff's alpha value of 0.586 indicates above average agreement between the three raters using ordinal evaluation metrics.

## Calculate Intra-Class Correlation (ICC)

The Intra-Class correlation (ICC) is another quantitative measure to assess the reliability of ratings by multiple subjects. The measure compares the variability of different ratings of the same subject to the total variation across all ratings and all subjects.

The [documentation of the pingouin library](https://pingouin-stats.org/build/html/generated/pingouin.intraclass_corr.html), which contains the Python implementation of ICC, can be referred to for more details.

In [14]:
# Function to calculate. Krippendorff's alpha to evaluate inter-rater reliability
def ICC(u1_rating_file_path, u2_rating_file_path, u3_rating_file_path):
    
    # Create file lists for Users 1 and 2 since they have similar rating file formats.
    u1_file_paths=glob.glob(u1_rating_file_path)
    u2_file_paths=glob.glob(u2_rating_file_path)
    file_paths=[u1_file_paths, u2_file_paths]
    user_df_dict={}
    
    # Loop through each file in the folders for Users 1 and 2.
    user_dict={}
    for file_path in file_paths:
        icc_dict={}
        for path in file_path:
            
            # Read each rating file as a dataframe and perform basic sanity checks.
            df=pd.read_excel(path)
            df.dropna(subset=["Query ID", "User ID"], inplace=True)
            # Ensure that each line item has product-level rating.
            df=df.loc[~(df["Detailed Rating (product level)"].isna())|(~(df["Relevance Indicator"].isna()))]
            # Ensure that the relevance indicator is present for at least one product for a given query and ranking method.
            df.loc[(df["product_id"].duplicated(keep=False))&(~(df["Detailed Rating (product level)"].isna())), "Relevance Indicator"]="Most relevant"
            df.dropna(subset=["Detailed Rating (product level)"], inplace=True)
            
            # Keep relevant columns.
            df=df.loc[:,["Query ID", "Ranking Method", "product_id", "User ID", "Detailed Rating (product level)"]]
            # Create a new column which is a string combination of Query ID, Ranking Method and product_id.
            df["Query ID__Ranking Method__product_id"]=df["Query ID"]+"_"+df["Ranking Method"]+"_"+df["product_id"]
            # Add relevant information as keys and values in a dictionary.
            icc_dict[df["User ID"].iloc[0]+"_"+df["Query ID"].iloc[0]+"_"+df["Ranking Method"].iloc[0]]=df
            
        # Add each user's dictionary as a value in a dictionary, creating a nested dictionary.
        user_dict[df["User ID"].iloc[0]]=icc_dict
    
    # Create a list of query IDs.
    queries=["Q1","Q2","Q3","Q4","Q5","Q6","Q7","Q8","Q9"]
    
    # Loop through each query ID for User 3, whose rating file format differs from those of Users 1 and 2.
    icc_dict={}
    for query in queries:
        
        # Read each sheet in the rating file as a dataframe and perform basic sanity checks.
        df=pd.read_excel(u3_rating_file_path, sheet_name=query)
        df.dropna(subset=["product_id","review_id"], inplace=True)

        # Separate the dataframe based on ranking method for ease of calculation.
        dfa=df.loc[df["Ranking Method"]=="Average"].copy()
        dfd=df.loc[df["Ranking Method"]=="Discounted Reward Only"].copy()
        dfda=df.loc[df["Ranking Method"]=="Discounted Reward with Adjustment"].copy()

        # Create a list of the dataframes corresponding to each ranking method.
        df_list=[dfa, dfd, dfda]

        # Loop through each dataframe in the list corresponding to a different ranking method.
        for dfx in df_list:
            
            # Keep relevant columns.
            dfx=dfx.loc[:,["Query ID", "Ranking Method", "product_id", "User ID", "Detailed Rating (product level)"]]
            # Create a new column which is a string combination of Query ID, Ranking Method and product_id.
            dfx["Query ID__Ranking Method__product_id"]=dfx["Query ID"]+"_"+dfx["Ranking Method"]+"_"+dfx["product_id"]
            # Add relevant information as keys and values in a dictionary.
            icc_dict[dfx["User ID"].iloc[0]+"_"+dfx["Query ID"].iloc[0]+"_"+dfx["Ranking Method"].iloc[0]]=dfx
    
    # Add User 3's dictionary as a value in the dictionary that contains the result dictionaries for Users 1 and 2.
    user_dict[dfx["User ID"].iloc[0]]=icc_dict
    
    # Combine all 3 users' dataframes by querying the nested dictionary.
    main_df=pd.DataFrame()
    for user in user_dict.keys():
        for df in user_dict[user]:
            dfx=user_dict[user][df]
            main_df=pd.concat([main_df, dfx])
    
    # Calculate ICC.
    icc=pg.intraclass_corr(data=main_df, targets="Query ID__Ranking Method__product_id", raters="User ID", 
                           ratings="Detailed Rating (product level)", nan_policy="omit").round(3)
    icc.set_index("Type")
    
    # Return the dataframe containing ICC values.
    return icc

In [15]:
icc=ICC(u1_rating_file_path, u2_rating_file_path, u3_rating_file_path)

In [16]:
icc

,Type,Description,ICC,F,df1,df2,pval,CI95%
0,ICC1,Single raters absolute,0.617,5.843,269,540,0.0,"[0.56, 0.67]"
1,ICC2,Single random raters,0.619,5.993,269,538,0.0,"[0.56, 0.68]"
2,ICC3,Single fixed raters,0.625,5.993,269,538,0.0,"[0.56, 0.68]"
3,ICC1k,Average raters absolute,0.829,5.843,269,540,0.0,"[0.79, 0.86]"
4,ICC2k,Average random raters,0.830,5.993,269,538,0.0,"[0.79, 0.86]"
5,ICC3k,Average fixed raters,0.833,5.993,269,538,0.0,"[0.8, 0.86]"


The above result table provides the ICC measures for various flavors, the definitions for which are detailed in the [Wikipedia page for ICC](https://en.wikipedia.org/wiki/Intraclass_correlation). In this case, we will be focusing on ICC3k, which is delineated by the following criteria:

Two-way mixed: k fixed raters are defined. Each subject is measured by the k raters.
Average measures: the reliability is applied to a context where measures of k raters will be averaged for each subject.
Consistency: in the context of repeated measurements by the same rater, systematic errors of the rater are cancelled and only the random residual error is kept.

For the interpretation of the ICC measure, there are two scales:

**Cicchetti scale:**

* Less than 0.40: poor.
* Between 0.40 and 0.59: fair.
* Between 0.60 and 0.74: good.
* Above 0.75: excellent.

Our ICC value of 0.834 implies that our three raters' inter-rater agreement is "excellent" on the Cicchetti scale with the 95% confidence intervals of 0.8 and 0.87 enabling it to stay within that category.

**Koo and Li scale:**

* Less than 0.50: poor
* Between 0.50 and 0.74: moderate
* Between 0.75 and 0.89: good
* Above 0.90: excellent

Our ICC value of 0.834 implies that our three raters' inter-rater agreement is "good" on the Koo and Li scale with the 95% confidence intervals of 0.8 and 0.87 enabling it to stay within that category.

### List of libraries used

In [17]:
session_info.show()